In [1]:
import pandas as pd
import json
import re

In [25]:
# data = pd.read_json('./mobile_devices.json')
data = pd.read_json('./mobile_devices.json')

In [3]:
data.columns

Index(['_id', 'name', 'mobile_specs', 'createdAt', 'original', 'slug',
       'category', 'image', 'img_uploaded', 'isActive', 'isNew', 'updatedAt',
       'description', 'fans', 'hits', 'keywords', 'popularity', 'subtitle',
       'brief_scrap', 'short_detail', 'views'],
      dtype='object')

Removing unnecessary features

In [4]:
data.drop(columns=[ 'createdAt' ,'original', 'slug', 'image', 'img_uploaded', 'isActive', 'isNew', 'updatedAt', 'fans', 'hits', 'keywords', 'description', 'popularity', 'subtitle', 'views'], inplace=True)

In [5]:
data['mobile_specs'][11]

[{'name': 'Network',
  'Technology': 'GSM / CDMA / HSPA / EVDO / LTE / 5G',
  '2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2',
  '3G bands': 'HSDPA 800 / 850 / 900 / 1700(AWS) / 1900 / 2100 ',
  '4G bands': '1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 17, 18, 19, 20, 26, 34, 38, 39, 40, 41',
  '5G bands': '1, 3, 28, 38, 40, 41, 77, 78, 79, 80, 84 SA/NSA',
  'Speed': 'HSPA 42.2/5.76 Mbps, LTE-A, 5G'},
 {'name': 'Launch',
  'Announced': '2021, February 22',
  'Status': 'Available. Released 2021, February 25'},
 {'name': 'Body',
  'Dimensions': 'Unfolded: 161.8 x 145.8 x 8.2 mmFolded: 161.8 x 74.6 x 14.7 mm',
  'Weight': '295 g (10.41 oz)',
  'Build': 'Glass front, glass back, aluminum frame',
  'SIM': 'Hybrid Dual SIM (Nano-SIM, dual stand-by)'},
 {'name': 'Display',
  'Type': 'Foldable OLED, 90Hz',
  'Size': '8.0 inches, 206.0 cm2 (~87.3% screen-to-body ratio)',
  'Resolution': '2200 x 2480 pixels (~413 ppi density)'},
 {'name': 'Platform',
  'OS': 'Android 10, EMUI 11, no Google Play Serv

set None if mobile specs have less then 10 objects

In [6]:
data['mobile_specs'] = data['mobile_specs'].apply(lambda x: x if len(x) > 10 else None)

In [7]:
data.dropna(inplace=True)

In [8]:
data.isna().sum()

_id             0
name            0
mobile_specs    0
category        0
brief_scrap     0
short_detail    0
dtype: int64

In [9]:
data.shape

(4081, 6)

handling _ID

In [10]:
data['_id'] = data['_id'].apply(lambda x: x['$oid'])

In [11]:
data['_id']

0       6341c19e4a92b2a60377c98a
1       6341c1a04a92b2a60377c98c
2       6341c1a24a92b2a60377c98e
3       6341c1c44a92b2a60377c990
4       6341c1d34a92b2a60377c992
                  ...           
4556    635fabf62203da0c176521f3
4557    635fabfc2203da0c176521f4
4558    635fac0c2203da0c176521f5
4560    635fac212203da0c176521f7
4561    635fac272203da0c176521f8
Name: _id, Length: 4081, dtype: object

handling mobile specs

In [12]:
for o in data['mobile_specs'].loc[1010]:
    print(o)

{'name': 'Network', 'Technology': 'GSM / HSPA / LTE', '2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2', '3G bands': 'HSDPA 850 / 900 / 1900 / 2100 ', '4G bands': '1, 3, 5, 7, 8, 34, 38, 39, 40, 41', 'Speed': 'HSPA 42.2/5.76 Mbps, LTE-A'}
{'name': 'Launch', 'Announced': '2019, May 28', 'Status': 'Available. Released 2019, June 01'}
{'name': 'Body', 'Dimensions': '156.7 x 74.3 x 8.8 mm (6.17 x 2.93 x 0.35 in)', 'Weight': '191 g (6.74 oz)', 'Build': 'Glass front (Gorilla Glass 5), glass back (Gorilla Glass 5), aluminum frame', 'SIM': 'Dual SIM (Nano-SIM, dual stand-by)'}
{'name': 'Display', 'Type': 'Super AMOLED, HDR10', 'Size': '6.39 inches, 100.2 cm2 (~86.1% screen-to-body ratio)', 'Resolution': '1080 x 2340 pixels, 19.5:9 ratio (~403 ppi density)', 'Protection': 'Corning Gorilla Glass 5'}
{'name': 'Platform', 'OS': 'Android 9.0 (Pie), upgradable to Android 10, MIUI 12', 'Chipset': 'Qualcomm SM8150 Snapdragon 855 (7 nm)', 'CPU': 'Octa-core (1x2.84 GHz Kryo 485 & 3x2.42 GHz Kryo

In [13]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [14]:
def handleSpecs(x):
    specs = []
    for o in x:
        if o['name'] == 'Battery':
            a = o.get('Charging')
            if a != None:
                specs.append(a.split(',')[0].lower().split('\n')[0])
            a = o.get('Type')
            if a != None:
                for i in a.split(','):
                    specs.append(i.replace('mAh', '').strip().replace(' ', '').lower())

        if o['name'] == 'Features':
            a = o.get('Sensors')
            if a != None:
                for i in a.split(','):
                    specs.append(i.split('(')[0].strip().replace(' ', ''))

        if o['name'] == 'Comms':
            a = o.get('USB')
            if a != None:
                specs.append(a.split(',')[0].replace(' ', '').split('.')[0].lower().replace('-', ''))
            a = o.get('WLAN')
            if a != None:
                for i in a.split(','):
                    f = (re.split(r'[a-z]/', i)[0]).strip().lower()
                    f = re.sub(' |-|.11', '', f)
                    if 'yes' in f or 'no' in f:
                        specs.append('wifi'+f)
                    else:
                        specs.append(f)

        if o['name'] == 'Selfie camera':
            try:t = list(o.keys())[1]
            except:continue
            a = o.get(t)
            if a != None:
                specs.append((f'{t}-' + a.split(',')[0].replace(' ', '')).lower())
        if o['name'] == 'Main Camera':
            try:t = list(o.keys())[1]
            except:continue
            a = o.get(t)
            if a != None:
                specs.append((f'{t}-' + a.split(',')[0].replace(' ', '')).lower())
        if o['name'] == 'Memory':
            a = o.get('Internal')
            if a != None:
                for x in a.split(','):
                    specs.append(x.replace('RAM', '').strip().replace(' ', '-'))

        if o['name'] == 'Platform':
            a = o.get('GPU')
            if a != None:
                specs.append("".join(a.split(' ')[0:2]).replace('-', '').lower())
            a = o.get('CPU')
            if a != None:
                specs.append(a.split(' ')[0].lower().replace('-core', ''))
                
            a = o.get('Chipset')
            if a != None:
                specs.append("".join(a.split(' ')[0:2]).lower())
            a = o.get('OS')
            if a != None:
                specs.append(a.split(',')[0].split('(')[0].replace(' ', '').lower())
        if o['name'] == 'Display':
            a = o.get('Resolution')
            if a != None:
                specs.append(a.split(',')[0].split('pixels')[0].replace(' ', '').lower())
            a = o.get('Type')
            if a != None:
                specs.append(a.split(',')[0].replace(' ', '').lower())
        if o['name'] == 'Body':
            a = o.get('Build')
            if a != None:
                for i in a.split(','):
                    specs.append(i.replace(' ', '').replace(')', '').replace('(', '').lower())

        if o['name'] == 'Network':
            a = o['Technology']
            specs.append(a.replace('/', '').replace(' ', '').lower())

        if o['name'] == 'Launch':
            a = o['Announced']
            if 'not' in a.lower():
                continue
            else:
                year = a.split(',')[0]
                month = ''
                for mo in months:
                    if mo in o['Announced']:
                        month = mo
                        break
                specs.append((year+month).lower())
    return specs 

In [15]:
data['specs'] = data['mobile_specs'].apply(handleSpecs)

In [16]:
data.drop(columns=['mobile_specs'], inplace=True)

In [17]:
data['displaySize'] = data['brief_scrap'].apply(lambda x: x['displaySize'])

In [18]:
data.drop(columns=['brief_scrap'], inplace=True)

In [19]:
data.drop(columns=['short_detail'], inplace=True)

In [20]:
data['tags'] = data['specs'].apply(lambda x: " ".join(x))

In [21]:
data['tags'] = data['tags'] + " " + data['category']

In [22]:
data['tags'][0]

'gsmhspalte 2021january glassfront plasticback plasticframe 720x1600 plslcd powervrge8100 quad mediatekmt6739w android10 32GB-2GB 32GB-3GB 32GB-4GB 64GB-3GB dual-13mp single-5mp microusb2 wifi802 wifidirect hotspot Accelerometer li-po5000 non-removable samsung'

In [23]:
data['displaySize'] = data['displaySize'].astype('string')

In [24]:
data['tags'] = data['tags'] + " " + data['displaySize']

In [25]:
data['tags'][0]

'gsmhspalte 2021january glassfront plasticback plasticframe 720x1600 plslcd powervrge8100 quad mediatekmt6739w android10 32GB-2GB 32GB-3GB 32GB-4GB 64GB-3GB dual-13mp single-5mp microusb2 wifi802 wifidirect hotspot Accelerometer li-po5000 non-removable samsung 6.5'

In [26]:
data.drop(columns=['category', 'specs', 'displaySize'], inplace=True)

In [27]:
data['tags'] = data['tags'] + " " + data['name'].str.lower()

stem the text to reduce

In [28]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [29]:
def stem(x):
    f = []
    for i in x.split():
        f.append(ps.stem(i))
    return " ".join(f)

In [30]:
data['tags'] = data['tags'].apply(stem)

text vectrization

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=4000, stop_words='english')

In [32]:
vectors = cv.fit_transform(data['tags']).toarray()

In [33]:
vectors.shape

(4081, 3987)

cosine similarity

In [34]:
from sklearn.metrics.pairwise import cosine_similarity
similairty = cosine_similarity(vectors)

In [35]:
similairty.shape

(4081, 4081)

In [36]:
similairty[1]

array([0.24733156, 1.        , 0.43954766, ..., 0.17609018, 0.37649664,
       0.26958193])

In [37]:
# data.iloc[0:10]

In [38]:
def recommendMe(phone):
    phone_index = data[data['name'].str.lower() == phone.lower()].index[0]
    distances = similairty[phone_index]
    phone_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:21]
    for x in phone_list:
        print(data.iloc[x[0]]['name'])
    

In [39]:
print(recommendMe('Samsung Galaxy A02'))
print(recommendMe('Realme C3'))
print(recommendMe('Sony Xperia Pro'))
print(recommendMe('ZTE Blade X1 5G'))
print(recommendMe('Nokia 1.4'))
print(recommendMe('Infinix Hot 10 Play'))

Samsung Galaxy A02s
Samsung Galaxy M02s
Samsung Galaxy M01
Samsung Galaxy A10s
Samsung Galaxy A11
Samsung Galaxy A21s
Samsung Galaxy J4
Samsung Galaxy A12
Samsung Galaxy M10
Samsung Galaxy A10
Samsung Galaxy J6
Samsung Galaxy A01
Infinix Hot 9
Samsung Galaxy J4+
Samsung Galaxy J6+
Realme C3
Samsung Galaxy Tab A 8.0 (2019)
Samsung Galaxy M01 Core
Samsung Galaxy M01s
Samsung Galaxy On6
None
Xiaomi Mi 10 5G
Xiaomi Mi 10 Ultra
Xiaomi Mi 9 Pro 5G
Xiaomi Mi 10T Pro 5G
Xiaomi Redmi K30 Pro Zoom
Xiaomi Mi 9 Pro
Xiaomi Black Shark 3 Pro
Xiaomi Poco F2 Pro
Xiaomi Redmi K30 Pro
Xiaomi Black Shark 3S
Xiaomi Mi 9 Explorer
Xiaomi Black Shark 3
Realme X50 Pro 5G
ZTE Axon 10s Pro 5G
Xiaomi Mi Note 10 Pro
Xiaomi Mi 10T 5G
Xiaomi Redmi K30S
vivo NEX 3S 5G
Meizu 17 Pro
Xiaomi Mi 11
None
Sony Xperia 1 II
Sony Xperia 5 II
Sony Xperia 5 Plus
Sony Xperia 5
Sony Xperia 1
Sony Xperia 10 III
Sony Xperia 10 II
Oppo Find X2 Pro
Sony Xperia XZ1 Compact
Sony Xperia L4
Sony Xperia H8541
Sony Xperia XZ2 Compact
Sony 